<div class="contentcontainer med left" style="margin-left: -50px;">
<dl class="dl-horizontal">
  <dt>Description</dt> <dd>Dynamic rendering quickstart guide</dd>
  <dt>Author</dt> <dd>Jean-Luc Stevens</dd>
  <dt>HoloViews</dt> <dd>>1.6.2</dd>
  <dt>Python</dt> <dd>2.7+</dd>
</dl>
</div>

This quickstart guide describes how to render visualizations dynamically instead of rendering all the necessary data before displaying it. This notebook uses core HoloViews together with the matplotlib backend.

In [ ]:
import holoviews as hv
import numpy as np
from holoviews.util import Dynamic
hv.notebook_extension()

Let's start by defining some simple NumPy arrays:

In [ ]:
ls = np.linspace(0, 10, 200)
xx, yy = np.meshgrid(ls, ls)

Using the ``xx`` and ``yy`` arrays, here is a simple function returning an ``hv.Image``:

In [ ]:
def cells(t):
    return hv.Image(np.sin(xx+t)*np.cos(yy+t))

We can now define a ``HoloMap`` to explore the effect of changing the ``t`` parameter:

In [ ]:
n = 10 # Number of images in the HoloMap
hv.HoloMap({t:cells(t) for t in np.linspace(0, np.pi/2, n)})

HoloMaps defined like this have the advantage of being statically exportable (if you save this notebook as HTML, the above cell and slider will work) but the disadvantage of requiring every image that can be displayed (i.e for every slider value) to be rendered before the output appears. If you change ``n`` above to be ``200``, you will notice it takes time before the output displays because matplotlib will be busy rendering all 200 images first. Once these images are all rendered, the slider is very responsive as all the data is already available.

## Using ``'live'`` mode for widgets

If you have a live notebook, it can be preferable to display the initial slider value as soon as it is available and have matplotlib render dynamically, as you move the slider around. The cell will display much quicker but generating each frame will take longer as matplotlib is kept busy rendering the data as it is requested.

To turn on this feature, use the output line magic as follows:

In [ ]:
%output widgets='live'

In [ ]:
n = 100 # Number of images in the HoloMap
hmap = hv.HoloMap({t:cells(t) for t in np.linspace(0, np.pi/2, n)})
hmap  # Slow if widgets mode is 'embed'!

Now you can have a HoloMap of 100 images that displays almost instantly and that you can explore by having matplotlib render each image only when necessary. Note that there are still 100 ``Image`` objects in this HoloMap that are created when the HoloMap is declared. In other words the ``cells`` function is called 100 times with different values of ``t`` before display occurs. The key point here is that these array operations are much faster than the corresponding matplotlib rendering process.

These 100 ``Image`` objects will also be consuming memory in the Python process which means such HoloMaps can still easily exhaust memory. In addition, when using ``%output widgets='live'``, static HTML export will only display the result at the initial slider position.

## Using ``util.Dynamic``

To start with, we will revert to the default 'embed' mode for widgets:

In [ ]:
%output widgets='embed'

HoloViews 1.6 introduced the ``DynamicMap`` and HoloViews 1.7 has introduced a way to generate ``DynamicMaps`` easily from HoloMaps:

In [ ]:
dmap = Dynamic(hmap)
dmap

The result is a ``DynamicMap`` that has many of the same advantages of of ``%output widgets='live'``:

In [ ]:
type(dmap)

The main advantage here is that you now have a handle on a ``DynamicMap`` object that you can use as input to a dynamic pipeline. You can apply indexing, slicing, selecting and composition operations to this DynamicMap which you couldn't do when using ``%output widgets='live'`` as that only affected the plotting process after the datastructures had been created.

## When to use each approach

* Use ``%output widgets='embed'`` when your data is small and quick to render. Use this when you wish to build reproducible documents that can be viewed as static HTML upon export.
*  Use ``%output widgets='live'`` when visualizations are taking too long to generate, if you are using a notebook based on ``HoloMaps`` (or compositions of ``HoloMaps``). Rendering will be dynamic but memory consumption will remain the same, so you may run out of memory with large ``HoloMap`` objects.  All ``HoloMap`` normalization options remain available, including ``-framewise`` (normalizing across all the elements of the HoloMap is possible as they all exist in Python's memory at once).
* Use ``Dynamic`` when building memory-efficient processing pipelines and require a ``HoloMap`` as input. This is useful when working in a notebook that emphasizes the use of ``DynamicMap``. Using ``DynamicMap`` objects can be even more memory efficient than ``Dynamic`` if it is possible to generate the data dynamically, but you may still need to interface with ``HoloMaps`` using ``Dynamic``. You will lose the ``-framewise`` normalization option, because only the elements already displayed can be normalized - most elements will not be in Python's memory (except for the elements that happen to be kept in the ``DynamicMap`` cache). Note that ``Dynamic`` may also be useful for building interactive applications based on the ``DynamicMap`` stream system.

As you can see there are various trade-offs to be made between reproducibility (i.e requirements for a live Python kernel versus static HTML) and the ability to rapidly explore a parameter space. When using ``DynamicMap`` for live rendering, real dynamic interactivity becomes possible using the [streams system](streams.ipynb) even using matplotlib as a backend. Using Bokeh as a plotting backend, you can use dynamic rendering using ``DynamicMaps`` together with streams to enable direct interaction with your visualizations.